In [3]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Simple EfficientNet-B0

In [4]:
# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [5]:
# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

In [8]:
# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

In [9]:
# 하이퍼파라미터 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_path = 'datasets_fin/'
model_name = 'efficientnet_b0'
img_size = 384
LR = 1e-3
EPOCHS = 30
BATCH_SIZE = 32
num_workers = 4

# 데이터 증강 설정
train_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [10]:
# 데이터 로드 및 분할
df = pd.read_csv("../data/train.csv")
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

train_dataset = ImageDataset(train_df, "../data/train_preprocessed/", transform=train_transform)
val_dataset = ImageDataset(val_df, "../data/train_preprocessed/", transform=val_transform)
test_dataset = ImageDataset("../data/sample_submission.csv", "../data/test_preprocessed/", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

In [11]:
# 모델 설정
model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [16]:
from torchsummary import summary

# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)
    
# 모델 구조 출력
print(f"\nModel structure of {model_name}:")
print_model_summary(model, (3, img_size, img_size))

# 모델 아키텍처 출력
print("\nModel architecture:")
print(model)


Model structure of efficientnet_b0:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 192, 192]             864
          Identity-2         [-1, 32, 192, 192]               0
              SiLU-3         [-1, 32, 192, 192]               0
    BatchNormAct2d-4         [-1, 32, 192, 192]              64
            Conv2d-5         [-1, 32, 192, 192]             288
          Identity-6         [-1, 32, 192, 192]               0
              SiLU-7         [-1, 32, 192, 192]               0
    BatchNormAct2d-8         [-1, 32, 192, 192]              64
            Conv2d-9              [-1, 8, 1, 1]             264
             SiLU-10              [-1, 8, 1, 1]               0
           Conv2d-11             [-1, 32, 1, 1]             288
          Sigmoid-12             [-1, 32, 1, 1]               0
    SqueezeExcite-13         [-1, 32, 192, 192]               0
  

In [17]:
# 학습 루프
best_val_f1 = 0
for epoch in range(EPOCHS):
    train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
    val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
    scheduler.step()

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_model.pth")

Loss: 0.5717: 100%|██████████| 40/40 [00:05<00:00,  7.09it/s]


Epoch 1/30
Train Loss: 0.9814, Train Acc: 0.6998, Train F1: 0.6748
Val Loss: 0.6068, Val Acc: 0.8248, Val F1: 0.8014


Loss: 0.2583: 100%|██████████| 40/40 [00:04<00:00,  8.28it/s]


Epoch 2/30
Train Loss: 0.3801, Train Acc: 0.8774, Train F1: 0.8633
Val Loss: 0.5170, Val Acc: 0.8599, Val F1: 0.8494


Loss: 0.2990: 100%|██████████| 40/40 [00:04<00:00,  8.40it/s]


...

Epoch 28/30
Train Loss: 0.0028, Train Acc: 0.9984, Train F1: 0.9985
Val Loss: 0.2432, Val Acc: 0.9427, Val F1: 0.9424


Loss: 0.0032: 100%|██████████| 40/40 [00:04<00:00,  8.34it/s]


Epoch 29/30
Train Loss: 0.0035, Train Acc: 0.9992, Train F1: 0.9993
Val Loss: 0.2408, Val Acc: 0.9427, Val F1: 0.9424


Loss: 0.0001: 100%|██████████| 40/40 [00:04<00:00,  8.32it/s]


Epoch 30/30
Train Loss: 0.0019, Train Acc: 1.0000, Train F1: 1.0000
Val Loss: 0.2418, Val Acc: 0.9427, Val F1: 0.9433


In [ ]:
# 테스트 데이터 추론
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
preds_list = []

for image, _ in tqdm(test_loader):
    image = image.to(device)
    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

In [ ]:
# 결과 저장
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv("pred.csv", index=False)
print("Prediction completed and saved to pred.csv")